In [ ]:
# !pip install -U pypdfium2
# # install package
# !pip install -U langchain-ollama

In [ ]:
# !pip install matplotlib
# !pip install pillow
# !pip install pdf2image

In [2]:
import os
import fitz  # PyMuPDF

def convert_pdf_to_img(pdf_path, drop_location):
    # Extract the PDF name without extension
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    
    # Ensure the drop location exists
    os.makedirs(drop_location, exist_ok=True)
    
    # Open the PDF
    doc = fitz.open(pdf_path)
    
    image_paths = []
    for i, page in enumerate(doc, start=1):
        # Generate image from page
        pix = page.get_pixmap()
        
        # Save the image as {pdf_name}_1.png, {pdf_name}_2.png, etc.
        img_path = os.path.join(drop_location, f"{i}.png")
        pix.save(img_path)
        image_paths.append(img_path)
    
    print(f"All images saved in: {drop_location}")

# Example usage
pdf_path = "3.pdf"           # Path to your PDF file
drop_location = "pdf3/"      # Folder where files should be saved
convert_pdf_to_img(pdf_path, drop_location)


All images saved in: pdf3/


### --- OCR ---

In [ ]:
!pip install pytesseract

In [10]:
from pytesseract import image_to_string 
from PIL import Image
from io import BytesIO

In [3]:
import os
from PIL import Image
from io import BytesIO
from pytesseract import image_to_string

def extract_text_from_images(folder_path):
    """
    Extracts text from all images in the specified folder.
    Processes images in numeric sequence if they are named accordingly.
    
    Args:
        folder_path (str): Path to the folder containing images.
    
    Returns:
        str: Combined text extracted from all images.
    """
    # List all files in the folder and sort them numerically (e.g., 1.png, 2.png)
    image_files = sorted(
        [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))],
        key=lambda x: int(os.path.splitext(x)[0])  # Extract numeric part for sorting
    )
    
    image_content = []
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        # Open the image file
        image = Image.open(image_path)
        
        # Use Tesseract to extract text
        raw_text = image_to_string(image)
        image_content.append(raw_text)
    
    # Join all extracted text with newline separators
    return "\n".join(image_content)

# Example usage
folder_path = "pdf3"  # Path to the folder containing images
extracted_text_pytessract = extract_text_from_images(folder_path)
print("Extracted Text:\n", extracted_text_pytessract)


Extracted Text:
 i

aT

PATIENT NAME : VAISHALI VAY BELLUBBI

TameNTID: #H.3991490) {QUENT PATENT 19: W380
AecasSion NO: 9081XDO12004 Act; 6OYeHrs Sex: Female DATEOF RTH: 26/02/1956
Dawn: 16/04/2026 02:36 ecerven: 16/04/2026 05:55 ePORTED: 16/04/2024 07:06
CQHENTRUNE = FHSL BG ROAD-190 REFERRING DOCTOR: OR. Rajpal RL. Singh

p:3981490 REQNO-17006613,
Wow ceutcieu
‘ab-ses10/24/1113

(axmmnew sad = capa ners erat

(£8625. EDTA WHOLE BLOOD

+ BL000 COUNTS, EDTA WHOLE BLOOD

HenOG.oBIN (HB) 3 120-150 va.

RED BLOOD CELL (RBC) COUNT 43 38-48 mv,

\WnITE at000 CEL. (w8C) COUNT 896 40-100 mow

PLATELET COUNT ss tow 180-410 thou
NETIC: AMOMTED aL COATED PRUE FONE (OC OTECION)

[RATELET COUNT COMTRMED ON SHEAR
‘Rac ANO PLATELET INDICES

reaToen Po as 46 *
MEM CORPUSCIUR VOUME DIC) #22 lm 29-1010 8
em CORPUSCIOR WEMOGLONNGHCH) 277 no-no =
we comuscuua wenoctoa 237 nisms wa

CONCENTRATION)

RED CEL DISTRIBUTION WIDTH (ROW) 2 116-140 *
MENTZER INDEX va
MEAN PLATELET VOLUME (HPV) ns
Wac DIFFERENTIA

In [9]:
# from PIL import Image
# from pytesseract import image_to_string
# from io import BytesIO

# def extract_text_with_pytesseract(image_source):
#     """
#     Extracts text from images using Tesseract OCR.
    
#     Parameters:
#     - image_source (str or list): A file path to a single image or a list of image bytes.

#     Returns:
#     - str: Extracted text from the image(s).
#     """
#     image_content = []

#     if isinstance(image_source, str):  # Single image file path
#         image = Image.open(image_source)
#         raw_text = str(image_to_string(image))
#         image_content.append(raw_text)

#     elif isinstance(image_source, list):  # List of image byte dictionaries
#         image_list = [list(data.values())[0] for data in image_source]
#         for index, image_bytes in enumerate(image_list):
#             image = Image.open(BytesIO(image_bytes))
#             raw_text = str(image_to_string(image))
#             image_content.append(raw_text)

#     else:
#         raise ValueError("Invalid input type. Provide a single file path or a list of dictionaries with image bytes.")

#     return "\n".join(image_content)


# # Example Usage with a Single Image File
# pdf1 = "1.png"
# text_with_pytesseract = extract_text_with_pytesseract(pdf1)
# print(text_with_pytesseract)



In [12]:
from langchain_ollama.llms import OllamaLLM
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate


llm= OllamaLLM(model="llama3.1:latest")

def convert_text_to_json_with_llm(extracted_text):
    """
    Uses an LLM to convert extracted text into JSON format.
    
    Parameters:
    - extracted_text (variable): A  detail text information identifiers and extracted text.

    Returns:
    - str: A JSON string with structured key-value pairs of every extracted information
    """
    prompt =  f"Return All textual Data in key Value Pair in JSON:\n{extracted_text}"

    response = llm(prompt)
    return response

In [ ]:
structured_json = convert_text_to_json_with_llm(extracted_text_pytessract)

print(structured_json)

In [4]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate
import json

def convert_text_to_json_with_llm(input_text):
    """
    Converts any input text into a structured JSON format.
    
    Parameters:
    - input_text (str): Any text input to be converted to JSON
    
    Returns:
    - str: A JSON string containing structured data
    """
    # Create a generic prompt template for JSON conversion
    template = """
    Convert the following text into a proper JSON format.
    Extract all relevant information as key-value pairs.
    Return ONLY a valid JSON object, nothing else.
    
    Text to convert:
    {text}
    
    Rules:
    1. Structure all information into logical key-value pairs
    2. Group related information together
    3. Use arrays for multiple related items
    4. Maintain proper JSON syntax
    5. Return only the JSON object with no additional text or explanation
    """
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["text"]
    )
    
    # Initialize the LLM
    llm = OllamaLLM(model="llama3.1:latest")
    
    # Generate the formatted prompt
    formatted_prompt = prompt.format(text=input_text)
    
    # Get response from LLM
    response = llm(formatted_prompt)
    
    try:
        # Clean the response
        cleaned_response = response.strip()
        # Find JSON boundaries
        start_idx = cleaned_response.find('{')
        end_idx = cleaned_response.rfind('}')
        
        if start_idx != -1 and end_idx != -1:
            cleaned_response = cleaned_response[start_idx:end_idx + 1]
            
            # Parse and validate JSON
            parsed_json = json.loads(cleaned_response)
            
            # Return formatted JSON string
            return json.dumps(parsed_json, indent=2, ensure_ascii=False)
        else:
            raise ValueError("No valid JSON object found in response")
            
    except Exception as e:
        raise Exception(f"Failed to parse LLM output into JSON: {str(e)}")


    
    
try:
        json_output = convert_text_to_json_with_llm(extracted_text_pytessract)
        json_output = json.loads(json_output)
        print(json_output)
except Exception as e:
        print(f"Error: {str(e)}")

/tmp/ipykernel_10887/1066867024.py:44: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_prompt)


{'Patient': {'Name': 'VAISHALI VAY BELLUBBI', 'Age': '68 Years', 'Gender': 'Female'}, 'Accession Numbers': ['00083XD012008', '00083XD012008', '00083XD012008'], 'Test Requested': {'Sample ID': '3991490', 'Request Number': '17006613', 'Date': '16/04/2028 02:36', 'Reception Date': '16/08/2028 05:55', 'Reported Date': '16/04/2026 07:06'}, 'Test Results': {'Status': 'Final', 'Biological Reference Interval Units': [{'Name': '', 'Units': ''}]}, 'Referring Doctor': {'Name': 'OR. Rajpal RL. Singh'}, 'Notes': {'Message': 'Please visit agillusdiagnostics.com for related test information for this accession.', 'Disclaimer': ['Test marked with "=" are outside the NABL accredited scope of the laboratory.', 'Sample received insufficient or inappropriate.', 'Specimen quality is unsatisfactory.', 'Incorrect specimen type.', 'Presumed that the Test sample belongs to the patient.', 'Test results may vary based on time of election, physiological condition of the patient, current medication or nutritional a

In [ ]:
print(type(json_output))

In [ ]:
json_output

### EasyOCR 

In [ ]:
!pip install easyocr

In [ ]:
from easyocr import Reader

# Load model for the English language
language_reader = Reader(["en"])

In [88]:
from easyocr import Reader
from PIL import Image
import numpy as np

def extract_text_with_easyocr(file_path):
    # Initialize EasyOCR reader
    language_reader = Reader(['en'], gpu=True)  # Adjust language or GPU usage as needed
    
    # Open the image file
    image = Image.open(file_path)
    
    # Convert the image to a NumPy array
    image_np = np.array(image)
    
    # Read text using EasyOCR
    raw_text = language_reader.readtext(image_np, detail=0)  # `detail=0` returns only the text
    raw_text = "\n".join(raw_text)
    
    return raw_text


In [ ]:
pdf1 = "1.png"
text_with_easyocr = extract_text_with_easyocr(pdf1)
print(text_with_easyocr)

try:
        json_output_easyocr = convert_text_to_json_with_llm(text_with_easyocr)
        json_output_easyocr = json.loads(json_output_easyocr)
        print(json_output_easyocr)
except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
print(json_output_easyocr)

In [ ]:
pdf1 = "1.png"
text_with_pytesseract = extract_text_with_pytesseract(pdf1)
print(text_with_pytesseract)

try:
        json_output_pytesseract = convert_text_to_json_with_llm(text_with_pytesseract)
        json_output_pytesseract = json.loads(json_output_pytesseract)
        print(json_output_pytesseract)
except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
print(json_output_pytesseract)

In [ ]:
print(json_output_easyocr)

### --------- Langchain ---------

In [ ]:
!pip install langchain

In [ ]:
# !pip install amazon-textract-caller
# !pip install amazon-textract-textractor

In [146]:
from langchain_community.document_loaders import AmazonTextractPDFLoader
loader = AmazonTextractPDFLoader("1.pdf")
documents = loader.load()

In [ ]:
documents[0]

In [ ]:
documents

In [144]:
# from langchain_community.document_loaders import AmazonTextractPDFLoader
# loader = AmazonTextractPDFLoader("3.pdf")
# documents = loader.load()

# # Multi Page Document to be on S3 Bucket 

## Upstage Extraction 

In [ ]:
pip install langchain_upstage

In [159]:
# pip install python-dotenv


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now you can access the environment variable
api_key = os.getenv("UPSTAGE_DOCUMENT_AI_API_KEY")

from langchain_upstage import UpstageLayoutAnalysisLoader

file_path = "3.pdf"

loader = UpstageLayoutAnalysisLoader(file_path,use_ocr=True,output_type="text")
data = loader.load()


In [ ]:
data

In [ ]:
!pip install llama-index-readers-nougat-ocr

In [ ]:
from llama_index.readers.nougat_ocr import PDFNougatOCR
from pathlib import Path

reader = PDFNougatOCR()

pdf_path = Path("1.png")

documents = reader.load_data(pdf_path)

In [9]:
documents

In [176]:
# !pip install google-generativeai

In [183]:
import google.generativeai as genai
import os
import base64

# os.environ['GOOGLE_API_KEY']
API_KEY = os.getenv('GOOGLE_API_KEY')

# api_key = os.getenv("UPSTAGE_DOCUMENT_AI_API_KEY")
genai.configure(api_key=API_KEY)

In [189]:

def prep_image(image_path):
    # Upload the file and print a confirmation.
    sample_file = genai.upload_file(path=image_path,
                                display_name="Diagram")
    print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")
    file = genai.get_file(name=sample_file.name)
    print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")
    return sample_file


def extract_text_from_image(image_path, prompt):
    # Choose a Gemini model.
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    # Prompt the model with text and the previously uploaded image.
    response = model.generate_content([image_path, prompt])
    return response.text
     

In [ ]:
my_api_key =os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=my_api_key)
sample_file = prep_image('1.png') 
text = extract_text_from_image(sample_file, "Extract the text in the image verbatim")

print(text)